In [1]:
import pandas as pd
import numpy as np
import json
import copy

In [24]:
output_bert = json.load(open('output_bert.json'))
output_bert

{'NLU00000': {'intent': 'Weather-Query',
  'slots': {'datetime_date': '前天', 'city': '平潭', 'index': '温度'}},
 'NLU00001': {'intent': 'Travel-Query',
  'slots': {'destination': '公厕', 'query_type': '导航'}},
 'NLU00002': {'intent': 'TVProgram-Play',
  'slots': {'channel': '中央3套', 'name': '艺喜乐汇'}},
 'NLU00003': {'intent': 'Alarm-Update',
  'slots': {'datetime_date': '周一', 'datetime_time': '10点'}},
 'NLU00004': {'intent': 'TVProgram-Play',
  'slots': {'name': ['国际足球赛场', '意甲联赛第38轮1'], 'datetime_date': '-20-21赛季'}},
 'NLU00005': {'intent': 'Travel-Query',
  'slots': {'departure': '上海', 'destination': '马来西亚', 'query_type': '机票查询'}},
 'NLU00006': {'intent': 'Alarm-Update',
  'slots': {'datetime_time': '刚才', 'notes': '去取快递吗'}},
 'NLU00007': {'intent': 'Weather-Query',
  'slots': {'city': '洛阳',
   'datetime_time': '今晚',
   'index': '穿衣指数',
   'type': '穿衣指数'}},
 'NLU00008': {'intent': 'Weather-Query',
  'slots': {'datetime_date': ['昨天', '天'], 'index': '湿度'}},
 'NLU00009': {'intent': 'Weather-Query',


In [25]:
output = json.load(open('output.json'))
output

{'NLU00000': {'intent': 'Weather-Query',
  'slots': {'datetime_date': '前天', 'city': '平潭', 'index': '温度'}},
 'NLU00001': {'intent': 'Travel-Query',
  'slots': {'destination': '公厕', 'query_type': '导航'}},
 'NLU00002': {'intent': 'TVProgram-Play',
  'slots': {'channel': '中央3套', 'name': '综艺喜乐汇'}},
 'NLU00003': {'intent': 'Alarm-Update',
  'slots': {'datetime_date': '周一', 'datetime_time': '10点'}},
 'NLU00004': {'intent': 'TVProgram-Play',
  'slots': {'name': '国际足球赛场-20-21赛季意甲联赛第38轮1'}},
 'NLU00005': {'intent': 'Travel-Query',
  'slots': {'departure': '上海', 'destination': '马来西亚', 'query_type': '机票查询'}},
 'NLU00006': {'intent': 'Alarm-Update', 'slots': {'notes': '去取快递'}},
 'NLU00007': {'intent': 'Weather-Query',
  'slots': {'city': '洛阳', 'datetime_time': '今晚', 'index': '穿衣指数'}},
 'NLU00008': {'intent': 'Weather-Query',
  'slots': {'datetime_date': ['昨天', '今天'], 'index': '湿度'}},
 'NLU00009': {'intent': 'Weather-Query',
  'slots': {'datetime_date': '今天', 'datetime_time': '中午', 'index': '紫外线指数'}}

In [26]:
test_a_text = json.load(open('test_A_text.json'))


In [27]:
data = {}
for idx, info_dict in output.items():
    if info_dict['intent'] == 'Other':
        continue
    if info_dict == output_bert[idx]:
        data[idx] = info_dict
        data[idx]['text'] = test_a_text[idx]['text']
        
data

{'NLU00000': {'intent': 'Weather-Query',
  'slots': {'datetime_date': '前天', 'city': '平潭', 'index': '温度'},
  'text': '前天平潭气温多少'},
 'NLU00001': {'intent': 'Travel-Query',
  'slots': {'destination': '公厕', 'query_type': '导航'},
  'text': '快点儿帮我找一下最近的公厕怎么走'},
 'NLU00003': {'intent': 'Alarm-Update',
  'slots': {'datetime_date': '周一', 'datetime_time': '10点'},
  'text': '现在给我定一个周一10点的闹钟'},
 'NLU00005': {'intent': 'Travel-Query',
  'slots': {'departure': '上海', 'destination': '马来西亚', 'query_type': '机票查询'},
  'text': '快点查下上海飞往马来西亚的航班哪天有票'},
 'NLU00009': {'intent': 'Weather-Query',
  'slots': {'datetime_date': '今天', 'datetime_time': '中午', 'index': '紫外线指数'},
  'text': '今天中午紫外线指数是否会继续上升'},
 'NLU00010': {'intent': 'HomeAppliance-Control',
  'slots': {'appliance': '空调', 'details': '调低', 'command': '风速调节'},
  'text': '能把空调风调低点吗'},
 'NLU00013': {'intent': 'FilmTele-Play',
  'slots': {'tag': '军事'},
  'text': '军事题材电视剧有吗播放一部让我感受一下'},
 'NLU00014': {'intent': 'Travel-Query',
  'slots': {'departure': '西安', 'de

In [28]:
def normalization(text):
    candidate_num_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    candidate_letter_list = [
        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
        'v', 'w', 'x', 'y', 'z',
        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
        'V', 'W', 'X', 'Y', 'Z'
    ]
    for i in candidate_num_list:
        text = text.replace(i, '*')
        
    for i in candidate_letter_list:
        text = text.replace(i, '@')
        
    return text

1720

In [38]:
# BIO 
nested_ner_num = 0
nested_ner_idx = []

seq_normalization_text = []
seq_text = []
seq_label = []
seq_intent = []
seq_idx = []
for idx, info_dict in data.items():
    intent = info_dict['intent']
    text = info_dict['text']
    if text in space:
        continue
    slots = info_dict['slots']
    if idx in ['NLU00247', 'NLU00149', 'NLU00247', 'NLU00294', 'NLU00479', 'NLU00722', 'NLU01044' \
              ,'NLU01203', 'NLU01233', 'NLU01344', 'NLU02386', 'NLU03108', 'NLU03302', 'NLU03829', \
              'NLU04934', 'NLU05234', 'NLU05801', 'NLU06413', 'NLU07935', 'NLU08272', 'NLU08449', \
              'NLU08552', 'NLU08880']:
        continue
    if text == '你好李焕英有了吧赶紧播放':
        continue
    default_label = len(text) * ['O']
    feasible_mark = True
    for slot_k, slot_value in slots.items():
        
        if slot_k in ['query_type', 'play_mode', 'command', 'index', 'language', 'type']:
            continue
            
            
        if slot_value == '银临，河图':
            slot_value = ['银临', '河图']
        elif slot_value == '那英，田震':
            slot_value = ['那英', '田震']
        elif slot_value == '谷枫，菲菲':
            slot_value = ['谷枫', '菲菲']
        elif slot_value == '姐姐 孤独的人是可耻的 蚂蚁':
            slot_value = ['姐姐', '孤独的人是可耻的', '蚂蚁']
        elif slot_value == '花粥，马玉阳':
            slot_value = ['花粥', '马玉阳']
        elif slot_value == '银临，云泣':
            slot_value = ['银临', '云泣']
        elif slot_value == '余生 小半':
            slot_value = ['余生', '小半']
        elif slot_value == '堕落天使 蕾丝花边':
            slot_value = ['堕落天使', '蕾丝花边']
        elif slot_value == '80年代 ':
            slot_value = '80年代'
        elif slot_value == '江伟麟，陈嘉慧':
            slot_value = ['江伟麟', '陈嘉慧']
        elif slot_value == '90年代 ':
            slot_value = '90年代'
        elif slot_value == '花粥，王胜男':
            slot_value = ['花粥', '王胜男']
        elif slot_value == '王菲，那英':
            slot_value = ['王菲', '那英']
        elif slot_value == '河图，洛天依':
            slot_value = ['河图', '洛天依']
        elif slot_value == '陈伟霆 ':
            slot_value = '陈伟霆'
        elif slot_value == '黄诗扶，B2ER':
            slot_value = ['黄诗扶', 'B2ER']
        elif text == '播放一个韩剧窥探吧' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '给我找一个美剧偶像剧播放一下吧' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '港片新喜剧之王播放一下可以吗' and slot_value == '香港':
            slot_value = '港片'
        elif text == '能找一个范迪塞尔导演的美剧看看么' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '你能找一个范迪塞尔拍摄的美剧给我看吗' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '给我播放韩剧宋仲基主演的太阳的后裔。' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '我想看一会港片新喜剧之王呢。' and slot_value == '香港':
            slot_value = '港片'
        elif text == '我想看一会港片新喜剧之王呢' and slot_value == '香港':
            slot_value = '港片'
        elif text == '泰剧禁忌女孩可以播放一下不' and slot_value == '泰国':
            slot_value = '泰剧'
        elif text == '我想看一会韩剧，那个朴信惠出演的可以吗' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '秋瓷炫演出的韩剧给我播放一个' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '范迪塞尔导演的美剧可以来一个看看不' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '张国荣的港片色情男女播放吧' and slot_value == '香港':
            slot_value = '港片'
        elif text == '我想看一个比较轻松快乐的美剧' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '你能找一个内地拍的神话剧看会吗古代的特效好点的' and slot_value == '大陆':
            slot_value = '内地'
        elif text == '美剧2009年上线的播放一下' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '美片指环王2播放一下给我看' and slot_value == '美国':
            slot_value = '美片'
        elif text == '你能给我播放一下韩剧W两个世界吗' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '韩剧顶楼可以播放出来吗' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '内地剧一起同过窗带有人生哲理，让我感受一下吧' and slot_value == '大陆':
            slot_value = '内地'
        elif text == '能来一个安妮海瑟薇的美剧看看吗' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '张根硕的韩剧可以找一个看不' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '是不是能找一个2018年的美剧看一下' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '我想看美剧悬疑片了就后裔' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '快点帮我播放韩剧请回答1988' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '能不能播放韩剧太阳的后裔给我看啊' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '罗晋唐嫣演的宫廷剧锦绣未央快放一下' and slot_value == ['唐嫣', '罗晋 ']:
            slot_value = ['唐嫣', '罗晋']
        elif text == '播放NBA全明星赛给我 ':
            text = '播放NBA全明星赛给我'


        if isinstance(slot_value, str):
            slot_value = [slot_value]
        for v in slot_value:
            try:
                start_idx = text.index(v)
            except Exception as e:
                print(slot_k)
                print(slot_value)
                print('=====')
                print(v)
                print(text)
                raise e
            if len(v) == 1:
                if default_label[start_idx] != 'O':
                    nested_ner_num += 1
                    nested_ner_idx.append(idx)
                    feasible_mark = False
                    break
                else:
                    default_label[start_idx] = 'B-' + str(intent) + '-' + str(slot_k)
#                     default_label[start_idx] = 'B-' + str(slot_k)

            else:
                if default_label[start_idx] != 'O':
                    nested_ner_num += 1
                    nested_ner_idx.append(idx)
                    feasible_mark = False
                    break
                else:
                    default_label[start_idx] = 'B-' + str(intent) + '-' + str(slot_k)
#                     default_label[start_idx] = 'B-' + str(slot_k)
                for extra_idx, char in enumerate(v[1:]):
                    if default_label[start_idx + 1 + extra_idx] != 'O':
                        nested_ner_num += 1
                        nested_ner_idx.append(idx)
                        feasible_mark = False
                        break
                    else:
                        default_label[start_idx + 1 + extra_idx] = 'I-' + str(intent) + '-' + str(slot_k)
#                         default_label[start_idx + 1 + extra_idx] = 'I-' + str(slot_k)

    if feasible_mark is True:
        seq_normalization_text.append(normalization(text))
        seq_text.append(text)
        seq_label.append(default_label)
        seq_intent.append(intent2idx[intent])
        seq_idx.append([label2idx[ll.split('-')[0]+'-'+intent+'-'+ll.split('-')[-1]] if ll != 'O' else label2idx[ll] for ll in default_label])

In [39]:
with open('/ai/223/person/lichunyu/datasets/dataf/seq_label/seq_label_test_a_labeled.train', 'w') as f:
    for idx in range(len(seq_normalization_text)):
        for char_idx in range(len(seq_normalization_text[idx])):
                if seq_normalization_text[idx][char_idx] and seq_normalization_text[idx][char_idx] != ' ':
                    f.write(seq_normalization_text[idx][char_idx] + '\t' + seq_label[idx][char_idx] + '\n')
        f.write('\n')

In [32]:
space = []
for i in seq_normalization_text:
    if ' ' in i:
        space.append(i)

In [19]:
IDX2LABEL = {
    0: '<pad>', 
    1: '<unk>', 
    2: 'O', 
    3: 'I-Video-Play-name', 
    4: 'I-Radio-Listen-channel', 
    5: 'I-Calendar-Query-datetime_date', 
    6: 'I-Alarm-Update-notes', 
    7: 'I-FilmTele-Play-name', 
    8: 'I-Alarm-Update-datetime_time', 
    9: 'I-Radio-Listen-name', 
    10: 'I-Alarm-Update-datetime_date', 
    11: 'I-HomeAppliance-Control-appliance', 
    12: 'I-Travel-Query-destination', 
    13: 'I-HomeAppliance-Control-details', 
    14: 'I-Radio-Listen-frequency', 
    15: 'I-Music-Play-song', 
    16: 'I-Weather-Query-datetime_date', 
    17: 'B-Calendar-Query-datetime_date', 
    18: 'I-Weather-Query-city', 
    19: 'B-HomeAppliance-Control-appliance', 
    20: 'B-Travel-Query-destination', 
    21: 'I-Video-Play-datetime_date', 
    22: 'I-Music-Play-artist', 
    23: 'B-Alarm-Update-datetime_date', 
    24: 'B-Weather-Query-city', 
    25: 'B-Video-Play-name', 
    26: 'B-Weather-Query-datetime_date', 
    27: 'B-Alarm-Update-datetime_time', 
    28: 'I-FilmTele-Play-artist', 
    29: 'B-Alarm-Update-notes', 
    30: 'B-HomeAppliance-Control-details', 
    31: 'B-FilmTele-Play-name', 
    32: 'B-Radio-Listen-channel', 
    33: 'I-Music-Play-age', 
    34: 'I-FilmTele-Play-age', 
    35: 'B-Radio-Listen-name', 
    36: 'I-FilmTele-Play-tag', 
    37: 'I-Music-Play-album', 
    38: 'B-Music-Play-artist', 
    39: 'B-FilmTele-Play-artist', 
    40: 'I-Travel-Query-departure', 
    41: 'B-Music-Play-song', 
    42: 'I-FilmTele-Play-play_setting', 
    43: 'I-Travel-Query-datetime_date', 
    44: 'B-Travel-Query-departure', 
    45: 'I-Radio-Listen-artist', 
    46: 'B-FilmTele-Play-tag', 
    47: 'I-Travel-Query-datetime_time', 
    48: 'B-Radio-Listen-frequency', 
    49: 'B-Radio-Listen-artist', 
    50: 'I-Video-Play-datetime_time', 
    51: 'B-Video-Play-datetime_date', 
    52: 'B-Travel-Query-datetime_date', 
    53: 'I-FilmTele-Play-region', 
    54: 'B-FilmTele-Play-region', 
    55: 'B-FilmTele-Play-play_setting', 
    56: 'I-TVProgram-Play-name', 
    57: 'B-FilmTele-Play-age', 
    58: 'B-Travel-Query-datetime_time', 
    59: 'B-Music-Play-age', 
    60: 'B-Music-Play-album', 
    61: 'I-Video-Play-region', 
    62: 'B-Video-Play-region', 
    63: 'I-Music-Play-instrument', 
    64: 'I-Weather-Query-datetime_time', 
    65: 'I-TVProgram-Play-channel', 
    66: 'B-Music-Play-instrument', 
    67: 'I-Audio-Play-name', 
    68: 'B-Video-Play-datetime_time', 
    69: 'B-Weather-Query-datetime_time', 
    70: 'B-TVProgram-Play-name', 
    71: 'I-TVProgram-Play-datetime_date', 
    72: 'I-Audio-Play-artist', 
    73: 'B-Audio-Play-name', 
    74: 'I-TVProgram-Play-datetime_time', 
    75: 'B-TVProgram-Play-channel', 
    76: 'B-TVProgram-Play-datetime_date', 
    77: 'B-Audio-Play-artist', 
    78: 'B-TVProgram-Play-datetime_time', 
    79: 'I-Audio-Play-play_setting', 
    80: 'B-Audio-Play-play_setting', 
    81: 'B-Audio-Play-tag', 
    82: 'I-Audio-Play-tag'
}

label2idx = {v: k-2 for k, v in IDX2LABEL.items() if v not in ['<pad>', '<unk>']}

In [20]:
INTENT = ['Music-Play',
        'HomeAppliance-Control',
        'Travel-Query',
        'Calendar-Query',
        'FilmTele-Play',
        'Weather-Query',
        'Video-Play',
        'Alarm-Update',
        'TVProgram-Play',
        'Audio-Play',
        'Radio-Listen',
        'Other']

intent2idx = {i: idx for idx, i in enumerate(INTENT)}
intent2idx

{'Music-Play': 0,
 'HomeAppliance-Control': 1,
 'Travel-Query': 2,
 'Calendar-Query': 3,
 'FilmTele-Play': 4,
 'Weather-Query': 5,
 'Video-Play': 6,
 'Alarm-Update': 7,
 'TVProgram-Play': 8,
 'Audio-Play': 9,
 'Radio-Listen': 10,
 'Other': 11}